In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensornetwork as tn

In [2]:
nCX = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
]).reshape(2, 2, 2, 2)

In [3]:
CX = tn.Node(nCX)

In [4]:
def untangle(P, T):    # (a_in, b_in, a_out, b_out)
    A, B, _ = tn.split_node(P, [P[0], P[2]], [P[1], P[3]])
    kd = tn.Node((np.array([1,0,0,0,0,0,0,1])*np.sqrt(2)).reshape(2,2,2))
    tn.connect(A[1], kd[0])
    tn.connect(B[2], T[1])
    tn.connect(kd[1], T[0])
    res = tn.contractors.auto([A, B, kd, T], output_edge_order=[A[0], B[1], kd[2], T[2]])
    return res


In [7]:
def create_random_unitary():
    # Create a random unitary 4x4 matrix
    U = np.random.rand(4, 4) # + 1j * np.random.rand(4, 4)
    U, _ = np.linalg.qr(U)
    return U.round(3)

In [8]:
target = tn.Node(create_random_unitary().reshape(2, 2, 2, 2))
for i in range(16):
    print(f"|{(i&0b1100)>>2:02b} {i&0b0011:02b}>", target.tensor.flatten()[i])

|00 00> -0.737
|00 01> 0.397
|00 10> 0.441
|00 11> 0.325
|01 00> -0.24
|01 01> -0.91
|01 10> 0.25
|01 11> 0.228
|10 00> -0.477
|10 01> -0.108
|10 10> -0.06
|10 11> -0.87
|11 00> -0.414
|11 01> -0.054
|11 10> -0.86
|11 11> 0.293


In [10]:
T = tn.Node(np.array([
    1, 0, 0, 1, 0, 1, 1, 0
]).reshape(2, 2, 2))
res = untangle(target, T)
print(f"rank: {np.linalg.matrix_rank(res.tensor.transpose(0, 2, 1, 3).reshape(4, 4))}")
res.tensor.reshape(4, 4)

rank: 4


array([[-1.0422754 ,  0.56144278,  0.45961941,  0.62366818],
       [-0.33941125, -1.28693434,  0.32244069,  0.35355339],
       [-0.67457987, -0.15273506, -1.2303658 , -0.08485281],
       [-0.58548441, -0.07636753,  0.41436457, -1.21622366]])

In [42]:
map(int,"1 0 0 1 1 0 0 1".split())